In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

24957


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
24952,2021-06-25,47,沖縄県,82,20472,4,179
24953,2021-06-26,47,沖縄県,74,20546,0,179
24954,2021-06-27,47,沖縄県,61,20607,0,179
24955,2021-06-28,47,沖縄県,25,20632,2,181
24956,2021-06-29,47,沖縄県,84,20716,0,181


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11682,9283,150
43,香川県,19647,2094,30
44,高知県,20709,1820,25
45,鳥取県,16461,470,2
46,鹿児島県,24426,3671,38


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
24950,2021-06-23,47,沖縄県,96,20328,0,171,568.0
24951,2021-06-24,47,沖縄県,62,20390,4,175,533.0
24952,2021-06-25,47,沖縄県,82,20472,4,179,529.0
24953,2021-06-26,47,沖縄県,74,20546,0,179,506.0
24954,2021-06-27,47,沖縄県,61,20607,0,179,506.0
24955,2021-06-28,47,沖縄県,25,20632,2,181,498.0
24956,2021-06-29,47,沖縄県,84,20716,0,181,484.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
24947,2021-06-20,47,沖縄県,61,20101,0,167,617.0,-0.371134,0.008904,-0.413462
24948,2021-06-21,47,沖縄県,33,20134,0,167,596.0,-0.459016,0.012012,-0.388889
24949,2021-06-22,47,沖縄県,98,20232,4,171,587.0,1.969697,0.153185,-0.084112
24950,2021-06-23,47,沖縄県,96,20328,0,171,568.0,-0.020408,0.139589,-0.165217
24951,2021-06-24,47,沖縄県,62,20390,4,175,533.0,-0.354167,0.111354,-0.360825
24952,2021-06-25,47,沖縄県,82,20472,4,179,529.0,0.322581,0.173637,-0.046512
24953,2021-06-26,47,沖縄県,74,20546,0,179,506.0,-0.097561,0.141427,-0.237113
24954,2021-06-27,47,沖縄県,61,20607,0,179,506.0,-0.175676,0.169350,0.000000
24955,2021-06-28,47,沖縄県,25,20632,2,181,498.0,-0.590164,0.150615,-0.242424
24956,2021-06-29,47,沖縄県,84,20716,0,181,484.0,2.360000,0.206372,-0.142857


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
530,2021-06-29,北海道,23,5,233.0,-0.148148,0.046518,-0.206897
1061,2021-06-29,青森県,11,0,37.0,4.500000,NaN,inf
1592,2021-06-29,岩手県,3,0,32.0,inf,NaN,-0.400000
2123,2021-06-29,宮城県,7,0,45.0,0.400000,0.062415,0.000000
2654,2021-06-29,秋田県,14,0,77.0,0.076923,0.389334,0.555556


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,12744,5252,111,2021-06-29,4,0,60.0,0.333333,0.829537,1.000000
1,京都府,13806,16574,242,2021-06-29,15,0,96.0,1.500000,0.264966,0.250000
2,佐賀県,21771,2555,24,2021-06-29,1,0,9.0,inf,NaN,0.000000
3,兵庫県,14868,40892,1288,2021-06-29,25,3,135.0,5.250000,0.627441,-0.074074
4,北海道,531,41294,1394,2021-06-29,23,5,233.0,-0.148148,0.046518,-0.206897


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,12744,5252,111,2021-06-29,4,0,60.0,0.333333,0.829537,1.000000,1813859
1,京都府,13806,16574,242,2021-06-29,15,0,96.0,1.500000,0.264966,0.250000,2545899
2,佐賀県,21771,2555,24,2021-06-29,1,0,9.0,inf,NaN,0.000000,823810
3,兵庫県,14868,40892,1288,2021-06-29,25,3,135.0,5.250000,0.627441,-0.074074,5549568
4,北海道,531,41294,1394,2021-06-29,23,5,233.0,-0.148148,0.046518,-0.206897,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,531,41294,1394,2021-06-29,23,5,233.0,-0.148148,0.046518,-0.206897,5267762,78.390026,783.900260,4.423131
41,青森県,1062,2490,31,2021-06-29,11,0,37.0,4.500000,NaN,inf,1275783,19.517426,195.174258,2.900180
19,岩手県,1593,1670,46,2021-06-29,3,0,32.0,inf,NaN,-0.400000,1235517,13.516609,135.166088,2.590009
11,宮城県,2124,9120,90,2021-06-29,7,0,45.0,0.400000,0.062415,0.000000,2292385,39.783893,397.838932,1.963021
36,秋田県,2655,862,20,2021-06-29,14,0,77.0,0.076923,0.389334,0.555556,985416,8.747575,87.475746,7.813959
15,山形県,3186,2020,47,2021-06-29,0,0,0.0,NaN,NaN,NaN,1082296,18.664025,186.640254,0.000000
35,福島県,3717,4868,161,2021-06-29,4,0,77.0,-0.692308,0.108615,-0.333333,1881981,25.866361,258.663610,4.091433
38,茨城県,4248,10517,164,2021-06-29,26,0,222.0,0.857143,0.201280,0.368421,2921436,35.999419,359.994195,7.599003
27,栃木県,4779,6996,81,2021-06-29,19,0,162.0,0.055556,-0.009019,-0.321429,1965516,35.593707,355.937067,8.242110
37,群馬県,5310,8032,152,2021-06-29,3,0,22.0,0.000000,NaN,-0.500000,1969439,40.783187,407.831875,1.117069


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      16908.297872
std       31093.316403
min         470.000000
25%        2522.500000
50%        5031.000000
75%       11001.500000
max      173220.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6903,173220,2231,2021-06-29,476,4,3464.0,0.501577,0.046520,0.094253,13834925,125.204871,1252.048710,25.038083
1,大阪府,14337,103275,2668,2021-06-29,101,7,686.0,1.525000,0.128088,-0.056075,8849635,116.699728,1166.997283,7.751732
2,神奈川県,7434,67181,948,2021-06-29,181,1,1421.0,-0.057292,0.021707,0.110429,9209442,72.947959,729.479593,15.429816
3,愛知県,12213,51040,961,2021-06-29,49,1,397.0,1.450000,0.042952,-0.489583,7575530,67.374824,673.748239,5.240557
4,埼玉県,5841,46326,831,2021-06-29,76,1,652.0,0.117647,-0.019065,-0.182796,7390054,62.686957,626.869574,8.822669
5,北海道,531,41294,1394,2021-06-29,23,5,233.0,-0.148148,0.046518,-0.206897,5267762,78.390026,783.900260,4.423131
6,兵庫県,14868,40892,1288,2021-06-29,25,3,135.0,5.250000,0.627441,-0.074074,5549568,73.685015,736.850148,2.432622
7,千葉県,6372,40132,705,2021-06-29,92,1,815.0,-0.239669,0.067084,0.243243,6319772,63.502291,635.022909,12.896035
8,福岡県,21240,35482,520,2021-06-29,26,0,213.0,0.529412,-0.006120,-0.277778,5129841,69.167836,691.678358,4.152175
9,沖縄県,24957,20716,181,2021-06-29,84,0,484.0,2.360000,0.206372,-0.142857,1481547,139.826816,1398.268162,32.668555


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      40.773733
std       29.626662
min        8.140446
25%       22.411168
50%       30.324114
75%       53.002691
max      139.826816
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,24957,20716,181,2021-06-29,84,0,484.0,2.360000,0.206372,-0.142857,1481547,139.826816,1398.268162,32.668555
26,東京都,6903,173220,2231,2021-06-29,476,4,3464.0,0.501577,0.046520,0.094253,13834925,125.204871,1252.048710,25.038083
32,神奈川県,7434,67181,948,2021-06-29,181,1,1421.0,-0.057292,0.021707,0.110429,9209442,72.947959,729.479593,15.429816
33,福井県,9558,1271,34,2021-06-29,12,0,116.0,0.500000,0.006828,-0.294118,780053,16.293765,162.937647,14.870784
5,千葉県,6372,40132,705,2021-06-29,92,1,815.0,-0.239669,0.067084,0.243243,6319772,63.502291,635.022909,12.896035
44,高知県,20709,1820,25,2021-06-29,10,0,71.0,-0.230769,0.040889,-0.375000,709230,25.661633,256.616330,10.010857
10,奈良県,15399,8186,129,2021-06-29,23,0,128.0,0.916667,0.053402,0.000000,1353837,60.465182,604.651816,9.454609
16,山梨県,10089,2097,21,2021-06-29,4,0,78.0,-0.333333,0.123771,-0.714286,826579,25.369626,253.696259,9.436485
7,埼玉県,5841,46326,831,2021-06-29,76,1,652.0,0.117647,-0.019065,-0.182796,7390054,62.686957,626.869574,8.822669
27,栃木県,4779,6996,81,2021-06-29,19,0,162.0,0.055556,-0.009019,-0.321429,1965516,35.593707,355.937067,8.242110


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    30.000000
mean      0.196846
std       0.277595
min      -0.060714
25%       0.026503
50%       0.087850
75%       0.234808
max       1.089161
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
29,滋賀県,13275,5551,90,2021-06-29,9,0,42.0,8.000000,1.089161,-0.307692,1420948,39.065469,390.654690,2.955773
0,三重県,12744,5252,111,2021-06-29,4,0,60.0,0.333333,0.829537,1.000000,1813859,28.954842,289.548416,3.307865
39,長崎県,22302,3178,69,2021-06-29,12,0,61.0,3.000000,0.717460,5.000000,1350769,23.527339,235.273389,4.515946
3,兵庫県,14868,40892,1288,2021-06-29,25,3,135.0,5.250000,0.627441,-0.074074,5549568,73.685015,736.850148,2.432622
36,秋田県,2655,862,20,2021-06-29,14,0,77.0,0.076923,0.389334,0.555556,985416,8.747575,87.475746,7.813959
40,長野県,10620,5031,91,2021-06-29,3,0,46.0,2.000000,0.362332,-0.571429,2087307,24.102827,241.028272,2.203797
1,京都府,13806,16574,242,2021-06-29,15,0,96.0,1.500000,0.264966,0.250000,2545899,65.100776,651.007758,3.770770
42,静岡県,11682,9283,150,2021-06-29,24,0,216.0,1.181818,0.244287,0.333333,3708556,25.031306,250.313060,5.824369
28,沖縄県,24957,20716,181,2021-06-29,84,0,484.0,2.360000,0.206372,-0.142857,1481547,139.826816,1398.268162,32.668555
38,茨城県,4248,10517,164,2021-06-29,26,0,222.0,0.857143,0.201280,0.368421,2921436,35.999419,359.994195,7.599003


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
21,広島県,18054,11486,175,2021-06-29,4,1,69.0,-0.500000,-0.060714,-0.666667,2826858,40.631684,406.316837,2.440873
7,埼玉県,5841,46326,831,2021-06-29,76,1,652.0,0.117647,-0.019065,-0.182796,7390054,62.686957,626.869574,8.822669
14,山口県,18585,3133,77,2021-06-29,0,0,17.0,-1.000000,-0.009524,-1.000000,1369882,22.870583,228.705830,1.240983
27,栃木県,4779,6996,81,2021-06-29,19,0,162.0,0.055556,-0.009019,-0.321429,1965516,35.593707,355.937067,8.242110
34,福岡県,21240,35482,520,2021-06-29,26,0,213.0,0.529412,-0.006120,-0.277778,5129841,69.167836,691.678358,4.152175
33,福井県,9558,1271,34,2021-06-29,12,0,116.0,0.500000,0.006828,-0.294118,780053,16.293765,162.937647,14.870784
17,岐阜県,11151,9256,186,2021-06-29,5,0,49.0,0.666667,0.020480,-0.722222,2032490,45.540199,455.401995,2.410836
32,神奈川県,7434,67181,948,2021-06-29,181,1,1421.0,-0.057292,0.021707,0.110429,9209442,72.947959,729.479593,15.429816
44,高知県,20709,1820,25,2021-06-29,10,0,71.0,-0.230769,0.040889,-0.375000,709230,25.661633,256.616330,10.010857
24,愛知県,12213,51040,961,2021-06-29,49,1,397.0,1.450000,0.042952,-0.489583,7575530,67.374824,673.748239,5.240557


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   470.    22063.75  43657.5   65251.25  86845.   108438.75 130032.5
 151626.25 173220.  ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([37.,  2.,  6.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 9., 23.,  3.,  9.,  0.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="富山県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
8486,2021-06-20,16,富山県,3,2009,0,37,50.0,-0.500000,0.066234,-0.400000
8487,2021-06-21,16,富山県,8,2017,0,37,53.0,1.666667,0.304329,0.600000
8488,2021-06-22,16,富山県,1,2018,0,37,49.0,-0.875000,0.179329,-0.800000
8489,2021-06-23,16,富山県,8,2026,0,37,52.0,7.000000,1.179329,0.600000
8490,2021-06-24,16,富山県,3,2029,0,37,44.0,-0.625000,0.918615,-0.727273
8491,2021-06-25,16,富山県,5,2034,0,37,34.0,0.666667,0.961905,-0.666667
8492,2021-06-26,16,富山県,2,2036,0,37,30.0,-0.600000,0.961905,-0.666667
8493,2021-06-27,16,富山県,1,2037,1,38,28.0,-0.500000,0.961905,-0.666667
8494,2021-06-28,16,富山県,0,2037,0,38,20.0,-1.000000,0.580952,-1.000000
8495,2021-06-29,16,富山県,3,2040,0,38,22.0,inf,NaN,2.000000
